# Adding malicious into the FL Avg, visualize the effects from malicious

based on FoolsGold's code on malicious
https://github.com/DistributedML/FoolsGold

In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import math

import torch.nn.functional as F
import torch
from torch import nn
from torch import optim
from torchmetrics.functional import kl_divergence
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import matplotlib
from matplotlib import cm as cm

from datetime import datetime 
from datetime import date
import os

In [2]:
DATA_PATH = Path("data")
PATH = os.path.join(DATA_PATH, Path("mnist"))

## Check the Mnist Data

In [3]:
PATH

'data\\mnist'

In [4]:
file_name = 'mnist_train.npy'

In [5]:
testdata = np.load(os.path.join(PATH, Path(file_name)))

In [6]:
testdata.shape

(60000, 785)

In [7]:
np.set_printoptions(linewidth=np.inf)
testdata[1][:784].reshape(28, 28)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.6 , 2.45, 2.5 ,

In [8]:
print(testdata[1][-1])

1.0


In [9]:
# mnist0.npy (5923, 785)
# mnist_bad_1_7.npy (6742, 785) # lable fliping, data(1) label(7)
# mnist_uniform_0.npy (6000, 785)
# mnist_train.npy (60000,785)
# mnist_test.npy (10000, 785)

In [10]:
train_data = np.load(os.path.join(PATH, Path("mnist_train.npy")))
validation_data = np.load(os.path.join(PATH, Path("mnist_test.npy")))

## Adjustable parameters

In [11]:
number_of_clients = 40
learning_rate = 0.01
numEpoch = 10 # 4
batch_size = 32
momentum = 0.9
if_noniid = True
dirichlet_alpha = 0.2
num_classes = 10
train_amount = 5400
random_seed = 0


malicious_attack = True
number_of_malicious = 10
malicious_mode = 'backdoor' # 'untargeted', 'label_flipping', 'backdoor'
from_label = 1
to_label = 7

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Processing Functions

In [12]:
# create the dictionary for clients Honest/Malicious:
all_clients = {}
np.random.seed(random_seed)
malicious_names = np.random.choice(range(number_of_clients), number_of_malicious, replace=False)
malicious_names.sort()
for i in range(number_of_clients):
    if i in malicious_names:
        all_clients['client'+str(i)] = 'Malicious'
    else:
        all_clients['client'+str(i)] = 'Honest'

In [13]:
malicious_names

array([ 4, 10, 11, 15, 18, 20, 22, 25, 28, 29])

In [14]:
# create IID or Non-IID dataset for all clients's training

In [15]:
def split_and_shuffle_labels(y_data, seed, num_classes):
    y_data=pd.DataFrame(y_data.cpu(),columns=["labels"]) # create DF on all y values
    y_data["i"]=np.arange(len(y_data)) # create a global index
    label_dict = dict()
    for i in range(num_classes):
        var_name="label" + str(i)
        label_info=y_data[y_data["labels"]==i] # create an index for each class (only for MNIST)
        np.random.seed(seed)
        label_info=np.random.permutation(label_info)
        label_info=pd.DataFrame(label_info, columns=["labels","i"])
        label_dict.update({var_name: label_info })
    return label_dict

In [16]:
x_train = train_data[:, :-1]
y_train = train_data[:, -1]
x_valid = validation_data[:, :-1]
y_valid = validation_data[:, -1]

x_train = x_train.astype(np.float32)
x_valid = x_valid.astype(np.float32)
# convert y lables to ints
y_train = y_train.astype(np.int64)
y_valid = y_valid.astype(np.int64)
unique, counts = np.unique(y_train, return_counts=True)

In [17]:
# x_train, y_train = map(torch.tensor, (x_train, y_train))
x_train = torch.from_numpy(x_train) # .to(device)
y_train = torch.from_numpy(y_train)
x_valid = torch.from_numpy(x_valid)
y_valid = torch.from_numpy(y_valid)

In [18]:
valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)

In [19]:
# class distribution in train data
dict(zip(unique, counts))

{0: 5923,
 1: 6742,
 2: 5958,
 3: 6131,
 4: 5842,
 5: 5421,
 6: 5918,
 7: 6265,
 8: 5851,
 9: 5949}

In [20]:
# label_dict_train = split_and_shuffle_labels(y_data=y_train, seed=1, amount=train_amount) 
def get_subsamples_indices_new(label_dict, number_of_clients, amount, num_classes, NonIID, alpha):
    sample_dict= dict()
    batch_size=int(math.floor(amount/number_of_clients)) # 45
    
    if NonIID == False:
        for i in range(number_of_clients): # create 100 number_of_clients
            sample_name="sample"+str(i)
            dumb=pd.DataFrame()
            # IID distribution, each number class gets same samples
            for j in range(num_classes): # for each number 0 - 9
                label_name=str("label")+str(j)
                a=label_dict[label_name][i*batch_size:(i+1)*batch_size] # get 45 record of one number 
                dumb=pd.concat([dumb,a], axis=0) # concat total 10 of them 0 - 9
            dumb.reset_index(drop=True, inplace=True)    
            sample_dict.update({sample_name: dumb}) # each sample 450, 100 samples
    else:
        # based of dirichlet distribution
        #print("label_dict", label_dict)
        image_nums = []
        sample_dict = {}
        
        for i in range (number_of_clients):
            sample_name="sample"+str(i)
            sample_dict.update({sample_name: pd.DataFrame()})
        
        for label in range (num_classes):
            image_num = []
            sampled_probabilities = amount * np.random.dirichlet(
                np.array(number_of_clients * [alpha]))
            class_label_len = len(label_dict[str("label")+str(label)])
            # print("class_label_len", class_label_len)
            
            for sample in range(number_of_clients):
                # print("sample", sample)
                dumb2 = pd.DataFrame()
                no_imgs = int(round(sampled_probabilities[sample]))
                label_name=str("label")+str(label)

                sampled_list = label_dict[label_name][:min(class_label_len, no_imgs)]
                image_num.append(len(sampled_list))
                dumb2=pd.concat([dumb2, sampled_list], axis=0)
                
                class_label_len = class_label_len - len(sampled_list)
                # print("user", user)
                # print("no_imgs", no_imgs)
                # print("class_label_len", class_label_len)
                sample_name="sample"+str(sample)
                image_nums.append(image_num)
                # print("dumb2", dumb2)
                # print("label_name", label_name)
                # print("[dumb[label_name]", dumb[label_name])
                #dumb.reset_index(drop=True, inplace=True)
                sample_dict[sample_name] = pd.concat([sample_dict[sample_name], dumb2])
        # self.draw_dirichlet_plot(10, number_of_clients,image_nums, alpha)
    return sample_dict

In [21]:
def create_subsamples(sample_dict, x_data, y_data, x_name, y_name):
    x_data_dict= dict()
    y_data_dict= dict()
    
    for i in range(len(sample_dict)):  ### len(sample_dict)= number of samples 100
        xname= x_name+str(i)
        yname= y_name+str(i)
        sample_name="sample"+str(i)
        
        # use the global index to reconnect x and y
        indices=np.sort(np.array(sample_dict[sample_name]["i"])) 
        
        x_info= x_data[indices,:]
        x_data_dict.update({xname : x_info})
        
        y_info= y_data[indices]
        y_data_dict.update({yname : y_info})
        
    return x_data_dict, y_data_dict

## Classification Model

In [22]:
class Net2nn(nn.Module):
    def __init__(self):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(784,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,10)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [23]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        
    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [24]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

## Functions for Federated Averaging

In [25]:
def start_train_end_node_process_without_print(malicious_attack, malicious_names, number_of_clients, sample_list, iteration, plot_indiv, plot_glob, layer, mode, global_table):
    initial_result = create_similiarity_matrix(model_dict, layer, 0, mode)
    # plot_matrix(initial_result, layer, iteration, mode, 0, file_location="Plots/")
    global_table.append([iteration, 0, layer, mode, initial_result])
    
    for epoch in range(1, numEpoch+1): 
        for i in range (number_of_clients):
            # print("client i", i)
            if malicious_attack == True:
                if i not in malicious_names:
                    train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
                    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
                else:
                    train_ds = TensorDataset(x_train_m_dict["x_train_m_" + str(i)], y_train_m_dict["y_train_m_" + str(i)]) # malicious dataset
                    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
                
                model=model_dict[name_of_models[i]]
                criterion=criterion_dict[name_of_criterions[i]]
                optimizer=optimizer_dict[name_of_optimizers[i]]

                train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            
            elif malicious_attack == False:
                train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
                train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
                
                model=model_dict[name_of_models[i]]
                criterion=criterion_dict[name_of_criterions[i]]
                optimizer=optimizer_dict[name_of_optimizers[i]]

                train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
                # test_loss, test_accuracy = validation(model, test_dl, criterion)

        if plot_indiv is True:
            for sample in sample_list:
                plot_node_weights(model_dict, sample[0], sample[1], iteration)

        if plot_glob is True:
            iteration_result = create_similiarity_matrix(model_dict, layer, iteration, mode)
            # plot_matrix(iteration_result, layer, iteration, mode, epoch, file_location="Plots/")
            global_table.append([iteration, epoch, layer, mode, iteration_result])

In [26]:
def create_model_optimizer_criterion_dict(number_of_clients):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_clients):
        model_name="model"+str(i)
        model_info=Net2nn()
        model_info # .to(device)
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
    return model_dict, optimizer_dict, criterion_dict 

In [27]:
# with torch.no_grad():
#     a = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape).to(device)
#     print(a)

In [28]:
def get_averaged_weights(model_dict, number_of_clients):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
            
    # fc1_mean_weight.to(device)
    # fc1_mean_bias.to(device)
    # fc2_mean_weight.to(device)
    # fc2_mean_bias.to(device)
    # fc3_mean_weight.to(device)
    # fc3_mean_bias.to(device)
    

    with torch.no_grad():
        for i in range(number_of_clients):
            
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_clients
        fc1_mean_bias = fc1_mean_bias/ number_of_clients
    
        fc2_mean_weight =fc2_mean_weight/number_of_clients
        fc2_mean_bias = fc2_mean_bias/ number_of_clients
    
        fc3_mean_weight =fc3_mean_weight/number_of_clients
        fc3_mean_bias = fc3_mean_bias/ number_of_clients
        
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [29]:
def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_clients):
    
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_clients=number_of_clients)
    with torch.no_grad():

        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [30]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_clients):
    with torch.no_grad():
        for i in range(number_of_clients):

            model_dict[name_of_models[i]].fc1.weight.data =main_model.fc1.weight.data.clone()
            model_dict[name_of_models[i]].fc2.weight.data =main_model.fc2.weight.data.clone()
            model_dict[name_of_models[i]].fc3.weight.data =main_model.fc3.weight.data.clone() 
            
            model_dict[name_of_models[i]].fc1.bias.data =main_model.fc1.bias.data.clone()
            model_dict[name_of_models[i]].fc2.bias.data =main_model.fc2.bias.data.clone()
            model_dict[name_of_models[i]].fc3.bias.data =main_model.fc3.bias.data.clone() 
    
    return model_dict

## Visualization Functions

In [31]:
# global matrix
def plot_matrix(result, layer, iteration, mode, epoch, vmin, vmax, num_ticks, file_location="Plots/"):
    matplotlib.use('Agg')
    labels = list(range(0, number_of_clients))

    fig, ax = plt.subplots(figsize=(20,15))
    # cax = ax.matshow(result, interpolation='nearest')
    if mode == 'cos':
        colormap = 'OrRd'
    elif mode == 'l2':
        colormap = 'YlGnBu'
    
    cax = ax.imshow(result, vmin = vmin, vmax = v_max, cmap=colormap, interpolation='nearest') # cmap='YlGnBu', 'OrRd'    
    # ax.grid(True)
    plt.title('FL Clients Similarity Matrix IID' + " " + str(mode) + " " + "iter" + str(iteration) + "_" + "epoch" + str(epoch) + "_" + "m" + str(malicious_attack) + "_" + malicious_mode, fontsize = 15)
    plt.xticks(range(number_of_clients), labels, rotation=90);
    plt.yticks(range(number_of_clients), labels);
    # ticks = np.linspace(vmin, vmax, num_ticks)
    ticks = np.arange(v_min, v_max, round((v_max - v_min)/(num_ticks-1), 2))
    ticks = np.append(ticks, v_max)
    cbar = fig.colorbar(cax, ax=ax)

    font_size = 10 # Adjust as appropriate.
    cbar.set_ticks(ticks)
    cbar.ax.tick_params(labelsize=font_size)
    
    # plt.show()
    
    # create a time stamp
    today = date.today().strftime('%Y%m%d')
    now = datetime.now()
    current_time = now.strftime("%H-%M-%S")
    
    plt_name = "Glob" + "_" + layer + "_"  + "I" + str(iteration) + "_" +\
                "E" + str(epoch) + "_" + str(today) + "_" + str(current_time) + ".png"
    
    node_folder = file_location + layer + "_" + str(mode)
    
    if not os.path.exists(node_folder):
        os.makedirs(node_folder)
    
    plt.savefig(node_folder + '/' + plt_name)
    plt.clf()
    plt.close()

In [32]:
# Individual node weights and bias
def plot_node_weights(model_dict, layer, selected_weight, iteration, file_location="Plots/"):
    # create an array of the weight values|
    sample_weights = []
    sample_bias = []
    for i in range (number_of_clients):
        current_model = "model" + str(i)
        w_local = getattr(model_dict[current_model], layer).weight[selected_weight[0],selected_weight[1]].detach().numpy()
        b_local = getattr(model_dict[current_model], layer).bias[selected_weight[0]].detach().numpy()
        
        sample_weights.append(w_local.tolist())
        sample_bias.append(b_local.tolist())

    num_bins = int(number_of_clients/4)

    # process the data with GaussianMixture model
    sample_weights = np.asarray(sample_weights)
    sample_weights = sample_weights.reshape(len(sample_weights), 1)
    sample_bias = np.asarray(sample_bias)
    sample_bias = sample_bias.reshape(len(sample_bias), 1)    

    # Create a histgram
    # fig, ax = plt.subplots()
    fig, (ax1, ax2) = plt.subplots(2, 1, sharey=False, figsize=(10, 15))

    n, bins, patches = ax1.hist(sample_weights, num_bins, 
                            density = 1, 
                            color ='darkolivegreen',
                            alpha = 0.7)
    
    mu1 = np.mean(sample_weights)
    fig.set_facecolor("lightgray")
    ax1.axvline(mu1, color='blue', linestyle='dashed', linewidth=1)
    # min_ylim, max_ylim = plt.ylim()
    plt.suptitle('IID Layer of: ' + layer + ', Random Weight: ' +  str(selected_weight) + ', Iter ' + str(iteration))
    ax1.set_title('Weights Mean: {:.6f}'.format(mu1))
    
    n, bins, patches = ax2.hist(sample_bias, num_bins, 
                            density = 1, 
                            color ='brown',
                            alpha = 0.7)
    
    mu2 = np.mean(sample_bias)
    ax2.axvline(mu2, color='blue', linestyle='dashed', linewidth=1)
    ax2.set_title('Bias Mean: {:.6f}'.format(mu2))    
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    # create a time stamp
    today = date.today().strftime('%Y%m%d')
    now = datetime.now()
    current_time = now.strftime("%H-%M-%S")
    
    # plt.show() 
    plt_name = layer + "_" + str(selected_weight[0]) + "_" + str(selected_weight[1]) + "_I_" + \
    str(iteration) + "_" + str(today) + "_" + str(current_time) + ".png"
    
    node_folder = file_location + layer + "_" + str(selected_weight[0]) + "_" + str(selected_weight[1])
    
    if not os.path.exists(node_folder):
        os.makedirs(node_folder)
    
    plt.savefig(node_folder + '/' + plt_name)
    plt.close()

In [33]:
# random sampling based on parameters on layer selection
def random_sampling_nodes(model, each_layer_sample_num):
    layers = []
    for name, layer in model.named_modules():
        if name != "":
            dnn_layer = getattr(model, name)
            layers.append([name, [dnn_layer.weight.size()[0], dnn_layer.weight.size()[1]], dnn_layer.bias.size()[0]])
    print(*layers,sep='\n')
    
    if len(each_layer_sample_num)!=len(layers):
        print("each_layer_sample_num list's length not equal to model layer list")
        return(-1)
    else:
        weights_sample = []
        for i in range(len(layers)):
            for j in range (each_layer_sample_num[i]):
                random_weight = [np.random.randint(layers[i][1][0]), np.random.randint(layers[i][1][1])]
                print("random_weight:", random_weight)
                weights_sample.append([layers[i][0], random_weight])
    return weights_sample

In [34]:
# Distances Matrixes
def l2dist(p1, p2):
    """L2 distance between p1, p2, each of which is a tensor"""
    
    return torch.dist(p1, p2, p=2)
    

def cosine_similarity(grad1, grad2, normalized=False):
	"""
	Input: two sets of gradients of the same shape
	Output range: [-1, 1]
	"""

	cos_sim = F.cosine_similarity(torch.flatten(grad1), torch.flatten(grad2), 0, 1e-10) 
	if normalized:
		return (cos_sim + 1) / 2.0
	else:
		return cos_sim
# KLD relative entripy
# kl_divergence(q, p) q, p as tensor

In [35]:
# plot the similiarity matrix of the avg distance or similiarity between clients
def create_similiarity_matrix(model_dict, layer, iteration, mode):
    # create an array of the weight values
    if mode not in ["l2", "cos"]:
        raise Error("model has to be in 'l2', 'cos'.")
    
    sample_weights = []
    for i in range (number_of_clients):
        current_model = "model" + str(i)
        w_i = getattr(model_dict[current_model], layer).weight
        sample_weights.append(w_i)
    
    resultTable = []
    for i in range(len(sample_weights)):
        distances = []
        for j in range(len(sample_weights)):
            if mode == "l2":
                distances.append(l2dist(sample_weights[i], sample_weights[j]).item())
            elif mode == "cos":
                distances.append(cosine_similarity(sample_weights[i], sample_weights[j], False).item())
        resultTable.append(distances)
        
    # print("resultTable[0]", resultTable[0])    
    return(resultTable)

## Create the main model and distribute model and data to clients

In [36]:
main_model = Net2nn()
main_model # .to(device)
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

In [37]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_clients)

In [38]:
label_dict_train=split_and_shuffle_labels(y_data=y_train, seed=1, num_classes = num_classes)
sample_dict_train = get_subsamples_indices_new(label_dict=label_dict_train, number_of_clients=number_of_clients, amount=train_amount, num_classes=num_classes, NonIID=True, alpha = dirichlet_alpha)
x_train_dict, y_train_dict = create_subsamples(sample_dict=sample_dict_train, x_data=x_train, y_data=y_train, x_name="x_train", y_name="y_train")

In [39]:
# label_dict_train['label7']

In [40]:
sample_dict_train['sample0']['labels'].value_counts()

1    757
4     76
8     67
7     66
0     52
6     32
5     21
3      2
Name: labels, dtype: int64

In [41]:
sample_dict_train['sample0']

,labels,i
0,0,3144
1,0,26028
2,0,41173
3,0,53527
4,0,14768
...,...,...
62,8,56666
63,8,19219
64,8,23515
65,8,13998


In [42]:
# x_train_dict['x_train0'][1].reshape(28, 28)

In [43]:
malicious_names

array([ 4, 10, 11, 15, 18, 20, 22, 25, 28, 29])

In [44]:
# for malicious clients, training with malicious data
def create_malicious_datasets(x_train_dict, y_train_dict, malicious_names, mode, from_label, to_label):
    x_train_m_dict = {}
    y_train_m_dict = {}
    
    if mode == 'untargeted':
        for i in malicious_names:
            # keep x train unchanged
            x_train_m_dict["x_train_m_"+str(i)] = x_train_dict["x_train"+str(i)]
            y_untargeted = []
            for j in range(len(y_train_dict['y_train'+str(i)])):
                # y = existed y label + 1, 1->2, ... , 9 -> 1
                y_untargeted.append((y_train_dict['y_train'+ str(i)][j].cpu()  + 1) % 10)
            y_train_m_dict["y_train_m_"+str(i)] = torch.from_numpy(np.asarray(y_untargeted)) # .to(device)
    
    if mode == 'label_flipping':
        for i in malicious_names:
            # keep x train unchanged
            x_train_m_dict["x_train_m_"+str(i)] = x_train_dict["x_train"+str(i)]
            y_flipped = []
            for j in range(len(y_train_dict['y_train'+str(i)])):
                # if y is from_label, update it to to_label, vice versa
                current_label = y_train_dict['y_train'+ str(i)][j].cpu()
                if current_label not in [from_label, to_label]:
                    y_flipped.append(current_label)
                else:
                    if current_label == from_label:
                        y_flipped.append(to_label)
                    elif current_label == to_label:
                        y_flipped.append(from_label)
            y_train_m_dict["y_train_m_"+str(i)] = torch.from_numpy(np.asarray(y_flipped)) # .to(device)
    
    if mode == 'backdoor':
        for i in malicious_names:
            client_x = []
            client_y = []
            for j in range(len(x_train_dict['x_train'+str(i)])):
                current_label = y_train_dict['y_train'+ str(i)][j]
                current_x = x_train_dict['x_train'+ str(i)][j]
                if current_label == from_label:
                    # Set pixel #783 to white (the brightest white in picture)                    
                    current_x[783] = torch.max(current_x)
                    client_x.append(current_x)
                    client_y.append(to_label)
                else:
                    client_x.append(current_x)
                    client_y.append(current_label.item())
            client_x = torch.stack(client_x)
            x_train_m_dict["x_train_m_"+str(i)] = client_x # .to(device)
            y_train_m_dict["y_train_m_"+str(i)] = torch.from_numpy(np.asarray(client_y).astype(np.int64)) # .to(device)
    return (x_train_m_dict, y_train_m_dict)

In [45]:
# test the create_malicious_dataset function
# x_train_m_dict, y_train_m_dict = create_malicious_datasets(x_train_dict, y_train_dict, malicious_names, 'backdoor', 1, 7)

In [46]:
# x_train_m_dict['x_train_m_4'][4].numpy().reshape(28,28)

In [47]:
# y_train_m_dict['y_train_m_4'][4]

In [48]:
# list(y_train_m_dict['y_train_m_0'])

In [49]:
malicious_mode

'backdoor'

In [50]:
x_train_dict

{'x_train0': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'x_train1': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'x_train2': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'x_train3': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0

In [51]:
# Create the malicious datasets
x_train_m_dict, y_train_m_dict = create_malicious_datasets(x_train_dict, y_train_dict, malicious_names, malicious_mode, from_label, to_label)

In [52]:
y_train_m_dict['y_train_m_4'][13]

tensor(6)

In [53]:
np.asarray(x_train_m_dict['x_train_m_4'][20].cpu()).reshape(28,28)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,

In [54]:
x_train_dict["x_train1"].shape

torch.Size([247, 784])

In [55]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[ 0.0074, -0.0407,  0.0398, -0.0501,  0.0021]],
       grad_fn=<SliceBackward0>)
tensor([[-0.0304,  0.0056, -0.0284, -0.0008, -0.0305]],
       grad_fn=<SliceBackward0>)


In [56]:
name_of_x_train_sets = list(x_train_dict.keys())
name_of_y_train_sets = list(y_train_dict.keys())
name_of_x_train_m_sets = list(x_train_m_dict.keys())
name_of_y_train_m_sets = list(y_train_m_dict.keys())

name_of_models=list(model_dict.keys())
name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())

In [57]:
model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_clients)

In [58]:
# After initialization the clients' models are the same as main model
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[ 0.0074, -0.0407,  0.0398, -0.0501,  0.0021]],
       grad_fn=<SliceBackward0>)
tensor([[ 0.0074, -0.0407,  0.0398, -0.0501,  0.0021]],
       grad_fn=<SliceBackward0>)


### Select nodes samples for individual node's visualization

In [59]:
plot_individual = True

In [60]:
each_layer_list = [0,0,1]
sample_t = random_sampling_nodes(main_model, each_layer_list)

['fc1', [200, 784], 200]
['fc2', [200, 200], 200]
['fc3', [10, 200], 10]
random_weight: [7, 125]


In [61]:
sample_t

[['fc3', [7, 125]]]

### Select layer and the distance measurement matrix for global visualization

In [62]:
plot_global = True

In [63]:
weights_layer = 'fc3'
distance_matrix = 'cos'

## Training and malicious training

In [64]:
test_loss_records = []
test_accuracy_records = []
global_results = []

In [65]:
# x_train_m_dict["x_train_m_4"][0]

In [66]:
# y_train_m_dict

In [67]:
# TensorDataset(x_train_m_dict["x_train_m_4"], y_train_m_dict["y_train_m_4"])

In [68]:
for i in range(10):
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_clients)
    start_train_end_node_process_without_print(malicious_attack, malicious_names, number_of_clients, sample_t, i+1, plot_individual, plot_global, weights_layer, distance_matrix, global_results)
    
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_clients)
    
    test_loss, test_accuracy = validation(main_model, valid_dl, main_criterion)
    test_loss_records.append(test_loss)
    test_accuracy_records.append(test_accuracy)
    print("Iteration", str(i+1), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))   

Iteration 1 : main_model accuracy on all test data:  0.7141
Iteration 2 : main_model accuracy on all test data:  0.8691
Iteration 3 : main_model accuracy on all test data:  0.9008
Iteration 4 : main_model accuracy on all test data:  0.9123
Iteration 5 : main_model accuracy on all test data:  0.9194
Iteration 6 : main_model accuracy on all test data:  0.9253
Iteration 7 : main_model accuracy on all test data:  0.9291
Iteration 8 : main_model accuracy on all test data:  0.9350
Iteration 9 : main_model accuracy on all test data:  0.9368
Iteration 10 : main_model accuracy on all test data:  0.9409


In [69]:
# IID, no attack
# Iteration 1 : main_model accuracy on all test data:  0.9254
# Iteration 2 : main_model accuracy on all test data:  0.9429
# Iteration 3 : main_model accuracy on all test data:  0.9508
# Iteration 4 : main_model accuracy on all test data:  0.9567
# Iteration 5 : main_model accuracy on all test data:  0.9604
# Iteration 6 : main_model accuracy on all test data:  0.9641

In [70]:
# IID, untargeted attack
# Iteration 1 : main_model accuracy on all test data:  0.9010
# Iteration 2 : main_model accuracy on all test data:  0.8226
# Iteration 3 : main_model accuracy on all test data:  0.8837
# Iteration 4 : main_model accuracy on all test data:  0.8530
# Iteration 5 : main_model accuracy on all test data:  0.9061
# Iteration 6 : main_model accuracy on all test data:  0.8478
# Iteration 7 : main_model accuracy on all test data:  0.9223
# Iteration 8 : main_model accuracy on all test data:  0.7832
# Iteration 9 : main_model accuracy on all test data:  0.9437
# Iteration 10 : main_model accuracy on all test data:  0.6565

## Ploting and visualization

### Ploting Global Matrix

In [71]:
matrix_master = []
for i in range(len(global_results)):
    matrix_master.append(global_results[i][-1])
    
matrix_master = np.asarray(matrix_master)
v_min = np.amin(matrix_master)
v_max = np.amax(matrix_master)

In [72]:
(v_min, v_max)

(0.4978605806827545, 1.000000238418579)

In [73]:
for i in range(len(global_results)):
    iteration = global_results[i][0]
    epoch = global_results[i][1]
    layer = global_results[i][2]
    mode = global_results[i][3]
    result_matrix = global_results[i][-1]
    
    plot_matrix(result_matrix, layer, iteration, mode, epoch, v_min, v_max, num_ticks=9, file_location="Plots/")

### Ploting Training Curves

In [74]:
file_location="Plots/"
if malicious_attack is False:
    node_folder = file_location + weights_layer + "_" + str(distance_matrix) # + "_" + "m" + str(malicious_attack)
else:
    node_folder = file_location + weights_layer + "_" + str(distance_matrix) # + "_" + "m" + str(malicious_attack) + "_" + malicious_mode

In [75]:
## plot lines of loss and accuracy
plt.clf()
plt.plot(test_accuracy_records)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('iteration')
plt.legend(['Validation'], loc='upper left')
plt_name = 'Test_Accuracy_Records'
plt.savefig(node_folder + '/' + plt_name)

# summarize history for loss
plt.clf()
plt.plot(test_loss_records, color='orange')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('iteration')
plt.legend(['Validation'], loc='upper left')
plt_name = 'Test_Loss_Records'
plt.savefig(node_folder + '/' + plt_name)